In [1]:
######JM Gawrych  11/30/2018 rev 3/11/2019
######B Snyder 6/26/2019 Added to IBM Watson Studio added ability to import arcgis Pythin libraries
######B SNyder 7/30/2019 No longer needed as of Pythin 3.6.  !pip install arcgis   #You must use this command the first time the notebook is run to load the arcgis module.  By using the '!' it works the same as using a command window but inside of Watson Studio.

#### Gets multipleThunderstorm Potential forecasts from severity 1 to 5.  Generates a geojson file from our featurizer with these 5 layers
#Use this link to see the results on ARCGIS Online: https://arcg.is/1zWbCW

import requests,json,datetime # need to install these libraries to run
#ESRI modules
from arcgis.gis import GIS
from arcgis import features
from arcgis.features import FeatureLayerCollection

##All TWC visual products have a product name and number
prod_num=str(44) # Radar Derived 24-Hour HailVision. 
prod_name='VAR0191209FROM25501surface12MinuteMaximum' # Per 
APIKEY='YOUR_TWC_KEY_HERE'  #This is IBM internal  key 


########################PRODUCT INVENTORY CALL################################  
def InvCall():
    base1 = 'https://api.weather.com/v2/tiler/info?products='+prod_num+':'+prod_name
    base2 = '&apiKey='+APIKEY
    url = base1 + base2 
    APIcall = requests.get(url) #Our API Call     to grab the latest forecast
    if APIcall.status_code == 200:  #if(APIcall.ok):
        data = APIcall.json()  # Just get most current run and next forecast
        t = data["layers"][prod_num][prod_name]["dimensions"][0]["t"][31]
        rt = data["layers"][prod_num][prod_name]["dimensions"][0]["rt"][0]
        print (rt,t)
        return [t, rt]
                    
##########################   API CALL   ###################################
def APIcall(t,rt,ts): # function for pulling data 
    count=0
    threshold = 1    
    thresholdS = str(threshold)   
    num_layers = 5
    inc=1
    
    fc = {
        "type": "FeatureCollection",
        "features": []
    }
   
              
    base1 = 'https://api.weather.com/v2/featurizer/feature?product='+prod_num+':'+prod_name
    base2 = '&t='+t+'&rt='+rt+ '&apiKey='+APIKEY
    base3= '&geometryType=polygon'

##add layer for lowest threshold and one for highest

    for x in range(0,num_layers):
        count=count+1
        base4 = '&operation=greaterThanOrEqual&threshold='+thresholdS

        url = base1 + base2 + base3 + base4  # make API URL   
        print (url)  
        data = requests.get(url).json() #Our API Call
        
        for y in data['features']:           
            f = {"type": "Feature", "properties": {}, "geometry": None}
            
            f['geometry'] = y['geometry']
            f['properties']['tStormIndex'] = y['properties']['threshold']
            f['properties']['Date'] = ts
            fc['features'].append(f)           
        
        threshold=threshold+inc
        thresholdS=str(threshold)                  
    
    with open("TstormIndex.geojson", "w") as outfile:
        json.dump(fc, outfile)
    
################################################################################
values = InvCall()
t= values[0]
rt=values[1]

now=datetime.datetime.utcnow()
year=str(now.year)
month=str('{:02d}'.format(now.month))
day=str('{:02d}'.format(now.day))
hour=str('{:02d}'.format(now.hour))
ts=year+"-"+month+"-"+day+","+hour+"00"

APIcall(t,rt,ts)

1564516800000 1564531200000
https://api.weather.com/v2/featurizer/feature?product=44:VAR0191209FROM25501surface12MinuteMaximum&t=1564531200000&rt=1564516800000&apiKey=c364a6ca121a4721a4a6ca121a072178&geometryType=polygon&operation=greaterThanOrEqual&threshold=1
https://api.weather.com/v2/featurizer/feature?product=44:VAR0191209FROM25501surface12MinuteMaximum&t=1564531200000&rt=1564516800000&apiKey=c364a6ca121a4721a4a6ca121a072178&geometryType=polygon&operation=greaterThanOrEqual&threshold=2
https://api.weather.com/v2/featurizer/feature?product=44:VAR0191209FROM25501surface12MinuteMaximum&t=1564531200000&rt=1564516800000&apiKey=c364a6ca121a4721a4a6ca121a072178&geometryType=polygon&operation=greaterThanOrEqual&threshold=3
https://api.weather.com/v2/featurizer/feature?product=44:VAR0191209FROM25501surface12MinuteMaximum&t=1564531200000&rt=1564516800000&apiKey=c364a6ca121a4721a4a6ca121a072178&geometryType=polygon&operation=greaterThanOrEqual&threshold=4
https://api.weather.com/v2/featurize